In [1]:
from data import CIFAR10, IMAGENETTE
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import utils
import copy

/Users/neddamj/miniforge3/envs/nhash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def decision_fn(orig_img, new_img, threshold=1):
    # Save the images and get their hashes
    orig_hash, new_hash = utils.compute_hash(orig_img), utils.compute_hash(new_img)
    # Make the decision based on the threshold
    hamming_dist = utils.distance(orig_hash, new_hash, 'hamming')
    if hamming_dist >= threshold:
        return True
    return False

In [3]:
def bin_boundary_search(orig_img, adv_img, l2_threshold, hamming_threshold):
    print('[INFO] Starting Boundary Search...')
    src_img = copy.deepcopy(orig_img)       # Copy of origin img that will be used for interpolation
    l2_dist = utils.distance(orig_img, adv_img, 'l2')
    while l2_dist >= l2_threshold:
        midpoint = (src_img + adv_img)/2
        if decision_fn(orig_img, midpoint, hamming_threshold):
            adv_img = midpoint
        else:
            src_img = midpoint
        print(f'L2 Dist: {l2_dist}')
        l2_dist = utils.distance(orig_img, adv_img, 'l2')
    print('[INFO] Boundary Search Complete...')
    return adv_img

In [4]:
img1, img2 = '../images/1.jpeg','../images/2.jpeg'
img = bin_boundary_search(img1, img2, 28, 2)
im = Image.fromarray(img.astype(np.uint8))
im.show()

[INFO] Starting Boundary Search...
Hamming Dist: 54
L2 Dist: 312.335916378285
Hamming Dist: 47
L2 Dist: 385.3155258334368
Hamming Dist: 19
L2 Dist: 192.6577629167185
Hamming Dist: 6
L2 Dist: 96.32888145835928
Hamming Dist: 0
L2 Dist: 48.164440729179695
Hamming Dist: 5
L2 Dist: 48.164440729179695
Hamming Dist: 5
L2 Dist: 36.12333054687674
Hamming Dist: 5
L2 Dist: 30.102775455734022
[INFO] Boundary Search Complete...


In [48]:
def estimate_grad_direction(img, sample_count):
    # Create the noise unit vectors
    noise = np.random.randn(sample_count, img.shape[0], img.shape[1], img.shape[2])
    noise /= np.linalg.norm(noise)
    # Get the signs of each img with the additive noise
    directions = np.zeros((noise.shape[0], 1))
    for i in range(noise.shape[0]):
        new = img/255 + noise[i]
        new = np.asarray(255*new).astype(np.uint8)
        np.clip(new, 0, 255)
        directions[i] = 1 if decision_fn(img, new, threshold=2) else -1
    # Compute the gradient direction estimate
    direction_estimate = sum([noise[i]*directions[i] for i in range(noise.shape[0])])
    return direction_estimate

In [49]:
def grad_based_update(orig_img, adv_img, grad_direction, stepsize, hamming_threshold):
    while True:
        new_img = img + stepsize*grad_direction
        if decision_fn(orig_img, adv_img, hamming_threshold):
            break
        stepsize /= 2
    return new_img

In [ ]:
def hsja(orig_img_path, 
         target_img_path,
         max_iters=10, 
         grad_queries=100, 
         l2_threshold=30, 
         hamming_threshold=5):
    orig_img, target_img = utils.load_img(orig_img_path), utils.load_img(target_img_path)
    for idx in range(1, max_iters+1):
        print(f'Iteration: {idx}')
        # Find the img as close to the boundary as possible
        boundary_img = bin_boundary_search(orig_img, target_img, l2_threshold, hamming_threshold)
        l2_dist = utils.distance(orig_img, boundary_img)
        if l2_dist < l2_threshold:
            print()
            return boundary_img
        # Estimate the gradient direction
        sample_count = int(grad_queries * (idx)**0.5)
        grad_direction = estimate_grad_direction(boundary_img, sample_count)
        # Calculate the stepsize
        stepsize = utils.distance(orig_img, boundary_img, 'l2')/np.sqrt(idx)
        target_img = grad_based_update(orig_img, boundary_img, grad_direction, stepsize, hamming_threshold)
    return target_img